# Práctica 1 opcional
Mejoras del programa: 
- Permitir crear figuras rellenas (seleccionar a través de la interfaz). 
- Poder crear otras primitivas (construir polilíneas y polígonos, elipses...) 
- Guardar el dibujo como un vídeo para ver cómo se realizó paso a paso 
- Aportación propia 

In [1]:
import cv2 as cv
import numpy as np
import os

os.makedirs("output", exist_ok=True)

file = 'images/eii.png'
image = cv.imread(file)
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")

original = image.copy()
canvas = image.copy()
drawing = False
ix, iy = -1, -1
points = []  # para polilíneas, polígonos y mano alzada

# Historial para deshacer/rehacer
history = [image.copy()]
history_index = 0

cv.namedWindow("Editor", cv.WINDOW_AUTOSIZE)

# Trackbars
def nothing(x): pass

cv.createTrackbar("Figura", "Editor", 0, 6, nothing)
cv.createTrackbar("B", "Editor", 0, 255, nothing)
cv.createTrackbar("G", "Editor", 0, 255, nothing)
cv.createTrackbar("R", "Editor", 0, 255, nothing)
cv.createTrackbar("Grosor", "Editor", 2, 20, nothing)
cv.createTrackbar("Relleno", "Editor", 0, 1, nothing)

def draw(event, x, y, flags, param):
    global ix, iy, drawing, canvas, image, points, history, history_index

    figura = cv.getTrackbarPos("Figura", "Editor")
    b = cv.getTrackbarPos("B", "Editor")
    g = cv.getTrackbarPos("G", "Editor")
    r = cv.getTrackbarPos("R", "Editor")
    color = (b, g, r)
    thickness = cv.getTrackbarPos("Grosor", "Editor")
    relleno = cv.getTrackbarPos("Relleno", "Editor")

    if figura not in [0,3,6] and relleno == 1: # relleno para figuras cerradas
        thickness = -1

    # Líneas, Rectángulos, Círculos, Elipses y Mano alzada
    if figura in [0,1,2,5,6]:
        if event == cv.EVENT_LBUTTONDOWN:
            drawing = True
            ix, iy = x, y
            if figura == 6:
                points = [(ix, iy)]
        elif event == cv.EVENT_MOUSEMOVE and drawing:
            temp = image.copy()
            if figura == 0:
                cv.line(temp, (ix, iy), (x, y), color, thickness)
            elif figura == 1:
                cv.rectangle(temp, (ix, iy), (x, y), color, thickness)
            elif figura == 2:
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv.circle(temp, (ix, iy), radius, color, thickness)
            elif figura == 5:
                center = ((ix + x) // 2, (iy + y) // 2)
                axes = (abs(x - ix) // 2, abs(y - iy) // 2)
                cv.ellipse(temp, center, axes, 0, 0, 360, color, thickness)
            elif figura == 6:
                cv.line(temp, points[-1], (x, y), color, thickness)
                points.append((x, y))
            canvas[:] = temp # copia los píxeles de temp en canvas (mismo objeto)
        elif event == cv.EVENT_LBUTTONUP:
            drawing = False
            if figura == 0:
                cv.line(image, (ix, iy), (x, y), color, thickness)
            elif figura == 1:
                cv.rectangle(image, (ix, iy), (x, y), color, thickness)
            elif figura == 2:
                radius = int(((x - ix) ** 2 + (y - iy) ** 2) ** 0.5)
                cv.circle(image, (ix, iy), radius, color, thickness)
            elif figura == 5:
                center = ((ix + x) // 2, (iy + y) // 2)
                axes = (abs(x - ix) // 2, abs(y - iy) // 2)
                cv.ellipse(image, center, axes, 0, 0, 360, color, thickness)
            elif figura == 6:
                for i in range(1, len(points)):
                    cv.line(image, points[i-1], points[i], color, thickness)
                points = []
            canvas[:] = image.copy()

            # Guardar estado en el historial
            if history_index < len(history) - 1:
                history = history[:history_index+1]  # eliminar posibles "redos"
            history.append(image.copy())
            history_index += 1

    # Polilíneas y polígonos
    elif figura in [3,4]:
        if event == cv.EVENT_LBUTTONDOWN:
            points.append((x,y))
        elif event == cv.EVENT_MOUSEMOVE and len(points)>0:
            temp = image.copy()
            preview_thickness = thickness if thickness>0 else 1
            cv.polylines(temp, [np.array(points)], False, color, preview_thickness) # líneas anteriores
            cv.line(temp, points[-1], (x,y), color, preview_thickness) # línea actual
            for p in points:
                cv.circle(temp, p, 4, (0,0,0), -1)
                cv.circle(temp, p, 2, (255,255,255), -1)
            canvas[:] = temp
        elif event == cv.EVENT_RBUTTONDOWN and len(points)>1:
            if figura == 3:
                cv.polylines(image, [np.array(points)], False, color, max(1,thickness)) # False: no une el punto inicial con el final
            elif figura == 4:
                if thickness == -1:
                    cv.fillPoly(image, [np.array(points)], color)
                else:
                    cv.polylines(image, [np.array(points)], True, color, thickness) # True: une el punto inicial con el final
            points = []
            canvas[:] = image.copy()

            # Guardar estado en el historial
            if history_index < len(history) - 1:
                history = history[:history_index+1]
            history.append(image.copy())
            history_index += 1

cv.setMouseCallback("Editor", draw)

# Vídeo
height, width = canvas.shape[:2]
fourcc = cv.VideoWriter_fourcc(*'XVID') # formato de vídeo
out = cv.VideoWriter('output/dibujo_1opcional.avi', fourcc, 20.0, (width, height))

print("Usa los sliders para cambiar figura, color, grosor y relleno.")
print("Figura: 0=Línea, 1=Rectángulo, 2=Círculo, 3=Polilínea, 4=Polígono, 5=Elipse, 6=Mano alzada")
print("Polilínea/polígono: clic izquierdo=punto, clic derecho=cerrar")
print("Tecla C = limpiar, Z = deshacer, Y = rehacer")
print("ESC para salir.")

while True:
    cv.imshow("Editor", canvas)
    out.write(canvas)
    key = cv.waitKey(1) & 0xFF
    if key == 27:  # ESC
        break
    elif key == ord('c'):  # limpiar canvas
        canvas[:] = original.copy()
        image[:] = original.copy()
        points = []
        drawing = False
        history = [image.copy()]
        history_index = 0
    elif key == ord('z'):  # deshacer
        if history_index > 0:
            history_index -= 1
            image[:] = history[history_index].copy()
            canvas[:] = image.copy()
    elif key == ord('y'):  # rehacer
        if history_index < len(history) - 1:
            history_index += 1
            image[:] = history[history_index].copy()
            canvas[:] = image.copy()

out.release()
cv.destroyAllWindows()


Usa los sliders para cambiar figura, color, grosor y relleno.
Figura: 0=Línea, 1=Rectángulo, 2=Círculo, 3=Polilínea, 4=Polígono, 5=Elipse, 6=Mano alzada
Polilínea/polígono: clic izquierdo=punto, clic derecho=cerrar
Tecla C = limpiar, Z = deshacer, Y = rehacer
ESC para salir.
